In [ ]:
pip install transformers accelerate


In [ ]:

!pip install -U bitsandbytes
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "ruslanmv/Medical-Llama3-8B"
device_map = "auto"  # You can adjust this if you want to manually assign layers to different devices

# Configure BitsAndBytes for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True  # Enable offloading to CPU when needed
)

# Load the model with quantization and CPU offloading
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_cache=False,
    device_map=device_map
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

def askme(question):
    sys_message = '''
    You are an AI Medical Assistant trained on a vast dataset of health information. Please be thorough and
    provide an informative answer. If you don't know the answer to a specific medical inquiry, advise seeking professional help.
    '''
    # Create the prompt by combining system message and user question
    prompt = sys_message + "\n\nQuestion: " + question + "\n\nAnswer:"

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

    # Generate the output from the model
    outputs = model.generate(**inputs, max_new_tokens=300, use_cache=True)  # Increased max_new_tokens

    # Decode and clean the output
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the answer from the response
    answer = response_text.split("Answer:")[-1].strip()
    return answer

# Test with a sample question
question = '''I'm a 35-year-old male and for the past few months, I've been experiencing fatigue,
increased sensitivity to cold, and dry, itchy skin. Could these symptoms be related to hypothyroidism? If so, what steps should I take to get a proper diagnosis and discuss treatment options?'''

print(askme(question))




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Hi there! As a trained AI Medical Assistant, I can provide some insight into your symptoms. It's possible that you may be experiencing symptoms of hypothyroidism, which is a condition where the thyroid gland doesn't produce enough hormones. This can cause a variety of symptoms, including fatigue, increased sensitivity to cold, and dry, itchy skin. If you're experiencing these symptoms, it's important to consult with a healthcare professional for a proper diagnosis and to discuss treatment options. In the meantime, you can try to manage your symptoms by staying hydrated, eating a balanced diet, and getting enough sleep. If your symptoms persist or worsen, it's important to seek medical attention as soon as possible. I hope this information is helpful! Please don't hesitate to ask for further clarification or guidance if needed.


In [ ]:
question = "tell me about anatomy"
print(askme(question))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The human body is a complex system of interconnected organs and tissues. It is important to understand the basics of anatomy to be able to provide accurate medical advice. Here is a brief overview of the main organs and systems in the human body:

    The brain is the control center of the body. It is responsible for processing information, making decisions, and controlling movement.
    The heart is a muscular organ that pumps blood throughout the body. It is located in the chest and is protected by the ribs.
    The lungs are a pair of organs that allow us to breathe. They are located in the chest and are protected by the ribs.
    The digestive system is responsible for breaking down food and absorbing nutrients. It includes the mouth, esophagus, stomach, intestines, and rectum.
    The circulatory system is responsible for transporting blood throughout the body. It includes the heart, blood vessels, and blood cells.
    The nervous system is responsible for transmitting information

In [ ]:
!pip install pyngrok


In [ ]:


!ngrok authtoken your token

from flask import Flask, request, jsonify
import subprocess

# Start ngrok programmatically with your auth token
def start_ngrok():
    from pyngrok import ngrok, conf
    conf.get_default().auth_token = "your token"  # Set your ngrok auth token
    http_tunnel = ngrok.connect(5000)
    print(f"Public URL: {http_tunnel.public_url}")

app = Flask(__name__)

@app.route('/ask', methods=['POST'])
def ask():
    try:
        # Parse the incoming JSON payload
        data = request.get_json()
        question = data.get('question', None)

        if not question:
            return jsonify({"error": "No question provided"}), 400

        # Generate the answer using the model
        answer = askme(question)
        return jsonify({"question": question, "answer": answer}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    start_ngrok()  # Start ngrok before running the Flask app
    app.run(port=5000)  # Ensure this matches the ngrok port


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: https://d4e0-34-83-141-102.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 16:20:27] "POST /ask HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 16:30:55] "POST /ask HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 16:31:42] "POST /ask HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 16:32:08] "POST /ask HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 16:48:38] "POST /ask HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:

ngrok process started


FileNotFoundError: [Errno 2] No such file or directory: 'nohup.out'